In [ ]:
!git clone https://github.com/huggingface/dataspeech.git
!cd dataspeech
!pip install --quiet -r ./dataspeech/requirements.txt

Cloning into 'dataspeech'...
remote: Enumerating objects: 650, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 650 (delta 138), reused 131 (delta 131), pack-reused 429 (from 1)
Receiving objects: 100% (650/650), 158.78 KiB | 15.88 MiB/s, done.
Resolving deltas: 100% (395/395), done.
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade protobuf wandb==0.16.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.6 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.6
    Uninstalling wandb-0.19.6:
      Successfully uninstalled wandb-0.19.6


In [ ]:
!git config --global credential.helper store
!huggingface-cli login --token YOUR_HF_TOKEN_HERE

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `SEGP-FINE-TUNING` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `SEGP-FINE-TUNING`


In [ ]:
%cd dataspeech

/content/dataspeech


GENERATE QUANTITATIVE ANNOTATIONS

In [ ]:
!python main.py "c0derish/master_emotion_audio_features_extracted" \
  --configuration "default" \
  --text_column_name "transcript" \
  --audio_column_name "filepath" \
  --cpu_num_workers 8 \
  --rename_column \
  --repo_id "c0derish/master_parler_tts_features_extracted" \
  --apply_squim_quality_estimation

GENERATE QUALITATIVE ANNOTATIONS (BIN VALUES FOR FEATURES)

Note: this overwrites the previous dataset

In [ ]:
!python ./scripts/metadataar_to_text.py \
    "c0derish/master_parler_tts_features_extracted" \
    --repo_id "master_parler_tts_features_extracted" \
    --configuration "default" \
    --cpu_num_workers 8 \
    --path_to_bin_edges "./examples/tags_to_annotations/v02_bin_edges.json" \
    --path_to_text_bins "./examples/tags_to_annotations/v02_text_bins.json" \
    --avoid_pitch_computation \
    --apply_squim_quality_estimation

README.md: 100% 764/764 [00:00<00:00, 6.36MB/s]
train-00000-of-00001.parquet: 100% 717k/717k [00:00<00:00, 10.4MB/s]
Setting num_proc from 8 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 100% 12162/12162 [00:00<00:00, 281243.63 examples/s]
Already computed bin edges have been passed for speaking_rate. Will use: [0.0, 3.8258038258038254, 7.651607651607651, 11.477411477411476, 15.303215303215302, 19.129019129019127, 22.95482295482295, 26.78062678062678].
Map (num_proc=8): 100% 12162/12162 [00:02<00:00, 5050.11 examples/s]
Already computed bin edges have been passed for noise. Will use: [17.12751579284668, 25.4012325831822, 33.67494937351772, 41.94866616385323, 50.22238295418875, 58.49609974452427, 66.76981653485979, 75.04353332519531].
Map (num_proc=8): 100% 12162/12162 [00:02<00:00, 4870.11 examples/s]
Already computed bin edges have been passed for reverberation. Will use: [10, 35, 45, 55, 59, 60].
Map (num_proc=8): 100%

GENERATE NATURAL LANGUAGE PROMPTS FROM BIN VALUES

In [ ]:
!python ./scripts/run_prompt_creation.py \
  --is_new_speaker_prompt \
  --dataset_name "c0derish/master_parler_tts_features_extracted" \
  --dataset_config_name "default" \
  --model_name_or_path "mistralai/Mistral-7B-Instruct-v0.2" \
  --per_device_eval_batch_size 128 \
  --attn_implementation "sdpa" \
  --output_dir "./master_emotion_audio_with_natural_language_prompts" \
  --load_in_4bit \
  --push_to_hub \
  --hub_dataset_id "master_emotion_audio_with_natural_language_prompts" \
  --preprocessing_num_workers 24 \
  --dataloader_num_workers 24

02/19/2025 01:40:47 - INFO - __main__ - *** Load annotated dataset ***
README.md: 100% 1.01k/1.01k [00:00<00:00, 7.60MB/s]
train-00000-of-00001.parquet: 100% 705k/705k [00:00<00:00, 9.11MB/s]
Setting num_proc from 24 back to 1 for the train split to disable multiprocessing as it only contains one shard.
02/19/2025 01:40:49 - WARNING - datasets.builder - Setting num_proc from 24 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 100% 12162/12162 [00:00<00:00, 272200.15 examples/s]
02/19/2025 01:40:49 - INFO - __main__ - *** Load pretrained model ***
config.json: 100% 596/596 [00:00<00:00, 4.85MB/s]
2025-02-19 01:40:51.224081: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 01:40:51.241865: E external/l

THATS ALL THE DATASET GENERATION DONE



NOW TO FINETUNE PARLER-TTS

In [ ]:
%cd ..
!git clone https://github.com/huggingface/parler-tts.git
%cd parler-tts
!pip install --quiet -e .[train]

/content
Cloning into 'parler-tts'...
remote: Enumerating objects: 1100, done.
remote: Counting objects: 100% (457/457), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 1100 (delta 344), reused 280 (delta 280), pack-reused 643 (from 2)
Receiving objects: 100% (1100/1100), 356.63 KiB | 16.98 MiB/s, done.
Resolving deltas: 100% (702/702), done.
/content/parler-tts
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!accelerate launch ./training/run_parler_tts_training.py \
    --model_name_or_path "parler-tts/parler-tts-mini-v1" \
    --feature_extractor_name "parler-tts/dac_44khZ_8kbps" \
    --description_tokenizer_name "parler-tts/parler-tts-mini-v1" \
    --prompt_tokenizer_name "parler-tts/parler-tts-mini-v1" \
    --overwrite_output_dir true \
    --train_dataset_name "c0derish/master_emotion_audio_features_extracted" \
    --train_metadata_dataset_name "c0derish/master_emotion_audio_with_natural_language_prompts" \
    --train_dataset_config_name "default" \
    --train_split_name "train" \
    --eval_dataset_name "c0derish/master_emotion_audio_features_extracted" \
    --eval_metadata_dataset_name "c0derish/master_emotion_audio_with_natural_language_prompts" \
    --eval_dataset_config_name "default" \
    --eval_split_name "train" \
    --max_eval_samples 16 \
    --per_device_eval_batch_size 8 \
    --target_audio_column_name "filepath" \
    --description_column_name "text_description" \
    --prompt_column_name "text" \
    --max_duration_in_seconds 20 \
    --min_duration_in_seconds 2.0 \
    --max_text_length 400 \
    --preprocessing_num_workers 2 \
    --do_train true \
    --num_train_epochs 4 \
    --gradient_accumulation_steps 12 \
    --gradient_checkpointing true \
    --per_device_train_batch_size 4 \
    --learning_rate 0.0003 \
    --adam_beta1 0.9 \
    --adam_beta2 0.99 \
    --weight_decay 0.018 \
    --lr_scheduler_type "constant_with_warmup" \
    --warmup_steps 50 \
    --logging_steps 2 \
    --freeze_text_encoder false \
    --audio_encoder_per_device_batch_size 8 \
    --dtype "float16" \
    --seed 456 \
    --output_dir "./output_dir_training/" \
    --temporary_save_to_disk "./audio_code_tmp/" \
    --save_to_disk "./tmp_dataset_audio/" \
    --dataloader_num_workers 8 \
    --do_eval \
    --predict_with_generate \
    --include_inputs_for_metrics \
    --group_by_length true

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-02-21 03:10:15.085434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740107415.108428   24367 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740107415.115048   24367 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Flash attention 2 is not installed
Using `include_inpu

In [ ]:
!pip install git+https://github.com/huggingface/parler-tts.git

  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-kuyp01wv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-kuyp01wv
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-0ji84k0o/descript-audiotools_22660a4852df4d5199b8f9a59a2a238a
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-0ji84k0o/descript-audiotools_22660a4852df4d5199b8f9a59a2a238a
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done


In [ ]:
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("./output_dir_training", torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")

prompt = "Hey, how are you doing today? The weather is so lovely its filling me with joy!"
description = "'A young woman delivers her words quite expressively and full of joy, in a very confined sounding environment with clear audio quality. She speaks fast.'"

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
model.push_to_hub("parler-tts-mini-v1-segp-colab-diff-hyperparams")
tokenizer.push_to_hub("parler-tts-mini-v1-segp-colab-diff-hyperparams")

model.safetensors:   0%|          | 0.00/1.76G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/c0derish/parler-tts-mini-v1-segp-colab-diff-hyperparams/commit/05dd1053161f3b8b7a9e75bc696758555b97adea', commit_message='Upload tokenizer', commit_description='', oid='05dd1053161f3b8b7a9e75bc696758555b97adea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/c0derish/parler-tts-mini-v1-segp-colab-diff-hyperparams', endpoint='https://huggingface.co', repo_type='model', repo_id='c0derish/parler-tts-mini-v1-segp-colab-diff-hyperparams'), pr_revision=None, pr_num=None)

In [ ]:
from IPython.display import Audio
Audio(audio_arr, rate=model.config.sampling_rate)

In [ ]:
prompt = "I just graduated from college! I'm so happy!"
description = "'A young man with a high pitched voice speaks quickly and in an elated manner in a confined space. His voice is very expressive and there is minimal noise.'"

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()

Audio(audio_arr, rate=model.config.sampling_rate)

In [ ]:
model = ParlerTTSForConditionalGeneration.from_pretrained("c0derish/parler-tts-mini-Jenny-colab").to(device)
tokenizer = AutoTokenizer.from_pretrained("c0derish/parler-tts-mini-Jenny-colab")

OSError: c0derish/parler-tts-mini-Jenny-colab is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`